### Load tensorflow

In [0]:
import tensorflow as tf

In [0]:
#Reset Default graph - Needed only for Jupyter notebook
tf.reset_default_graph()

### Collect Data

In [0]:
import pandas as pd

In [4]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [5]:
data = pd.read_csv('/content/gdrive/My Drive/AIML/Labs/R6-Internal/prices.csv')
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


### Check all columns in the dataset

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


### Drop columns `date` and  `symbol`

In [0]:
data = data.drop(columns=['date','symbol'], axis=1)

In [8]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [9]:
data1000 = data.head(1000)
data1000.shape

(1000, 5)

### Divide the data into train and test sets

In [10]:
from sklearn.model_selection import train_test_split

X = data1000
y = data1000['close']
X = X.drop(columns='close', axis=1)

print(X.shape)
print(y.shape)

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3, random_state=7)

(1000, 4)
(1000,)


In [11]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(700, 4)
(700,)
(300, 4)
(300,)


In [12]:
X.head()

,open,low,high,volume
0,123.430000,122.309998,126.250000,2163600.0
1,125.239998,119.940002,125.540001,2386400.0
2,116.379997,114.930000,119.739998,2489500.0
3,115.480003,113.500000,117.440002,2006300.0
4,117.010002,114.089996,117.330002,1408600.0


In [13]:
y.head()

0    125.839996
1    119.980003
2    114.949997
3    116.620003
4    114.970001
Name: close, dtype: float64

In [14]:
data1000.shape

(1000, 5)

## Building the graph in tensorflow

1.Define input data placeholders

In [0]:
x = tf.placeholder(dtype=tf.float32, shape=[None,4], name='x-input')
x_n = tf.layers.batch_normalization(x,training=True)
y_ = tf.placeholder(dtype=tf.float32, shape=[None], name='y-input')

2.Define Weights and Bias

In [0]:
W = tf.Variable(tf.zeros(shape=[4,1]), dtype=tf.float32, name="Weights")
b = tf.Variable(tf.zeros(shape=[1]), name="Bias")

3.Prediction

In [0]:
y = tf.add(tf.matmul(x_n, W), b, name='output')

4.Loss (Cost) Function [Mean square error]

In [0]:
loss = tf.reduce_mean(tf.square(y-y_), name='Loss')

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [0]:
train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

## Execute the Graph for 100 epochs and observe the loss

In [0]:
sess = tf.Session()

sess.run(tf.global_variables_initializer())

training_epochs = 100

In [21]:
for epoch in range(training_epochs):
  
  _, train_loss = sess.run([train_op,loss], feed_dict={x:X_train,
                                                       y_:y_train})
  
  if epoch % 10 == 0:
     print ('Training loss at step: ', epoch, ' is ', train_loss)

('Training loss at step: ', 0, ' is ', 8370.378)
('Training loss at step: ', 10, ' is ', 6810.4746)
('Training loss at step: ', 20, ' is ', 5374.9844)
('Training loss at step: ', 30, ' is ', 3678.1487)
('Training loss at step: ', 40, ' is ', 3677.587)
('Training loss at step: ', 50, ' is ', 3677.599)
('Training loss at step: ', 60, ' is ', 3677.6038)
('Training loss at step: ', 70, ' is ', 3677.5989)
('Training loss at step: ', 80, ' is ', 3677.6008)
('Training loss at step: ', 90, ' is ', 3677.6023)


In [22]:
train_loss

3677.598

### Get the shapes and values of W and b

Hint: Use sess.run(W) to get W.

In [23]:
sess.run(W)

array([[-4.726437  ],
       [-1.8942949 ],
       [ 4.269225  ],
       [-0.03930737]], dtype=float32)

In [24]:
sess.run(b)

array([24.23879], dtype=float32)

### Find the Absolute mean square loss difference between training and testing loss.

In [0]:
Y_pred = sess.run(y, feed_dict={x:X_test})

In [26]:
y_test.dtype

dtype('float64')

In [27]:
from sklearn.metrics import mean_squared_error
test_loss = mean_squared_error(y_test,Y_pred)
test_loss

2770.02488656748

In [0]:
##tf.reduce_mean(tf.square(Y_test-Y_pred))

In [29]:
difference = train_loss - test_loss
difference

907.5730138231452

In [30]:
'''for epoch in range(training_epochs):
  
  _, test_loss_1 = sess.run([train_op,loss], feed_dict={x:X_test,
                                                       y_:Y_test})
  
  if epoch % 10 == 0:
     print ('Test loss at step: ', epoch, ' is ', test_loss_1)'''

"for epoch in range(training_epochs):\n  \n  _, test_loss_1 = sess.run([train_op,loss], feed_dict={x:X_test,\n                                                       y_:Y_test})\n  \n  if epoch % 10 == 0:\n     print ('Test loss at step: ', epoch, ' is ', test_loss_1)"

<h4> The difference between training and testing loss is 907.57 <h4>

### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [0]:
tf.reset_default_graph()
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Reshape data from 2D to 1D -> 28x28 to 784
#model.add(tf.keras.layers.Reshape(4000,),input_shape=(1000,4))

#Normalize the data
model.add(tf.keras.layers.BatchNormalization(input_shape =(4,)))

#Add Dense Layer which provides 10 Outputs after applying softmax
model.add(tf.keras.layers.Dense(1))

#Comile the model
model.compile(optimizer='sgd', loss='mse')

In [32]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 4)                 16        
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 21
Trainable params: 13
Non-trainable params: 8
_________________________________________________________________


### Execute the model

In [33]:
model.fit(X_train,y_train, validation_data=(X_test,y_test), epochs=100, batch_size=X_train.shape[0])

Train on 700 samples, validate on 300 samples
Epoch 1/100
700/700 [==============================] - 0s 199us/step - loss: 8400.7217 - val_loss: 1608.4996
Epoch 2/100
700/700 [==============================] - 0s 5us/step - loss: 7756.4233 - val_loss: 3882.7837
Epoch 3/100
700/700 [==============================] - 0s 8us/step - loss: 4895.8193 - val_loss: 290804.8125
Epoch 4/100
700/700 [==============================] - 0s 4us/step - loss: 441.3973 - val_loss: 16317.4199
Epoch 5/100
700/700 [==============================] - 0s 3us/step - loss: 2476.3733 - val_loss: 352788.5312
Epoch 6/100
700/700 [==============================] - 0s 4us/step - loss: 3990.4280 - val_loss: 3858.4458
Epoch 7/100
700/700 [==============================] - 0s 5us/step - loss: 6481.1558 - val_loss: 13604.5752
Epoch 8/100
700/700 [==============================] - 0s 4us/step - loss: 1065.9459 - val_loss: 219207.7656
Epoch 9/100
700/700 [==============================] - 0s 8us/step - loss: 3209.2808 - va

In [34]:
type(X_train)

pandas.core.frame.DataFrame

In [35]:
type(y_train)

pandas.core.series.Series

### Classification using Keras 

### Load the given Iris data using pandas (Iris.csv)

In [1]:
import pandas as pd
iris_df = pd.read_csv('/content/gdrive/My Drive/AIML/Labs/R6-Internal/Iris.csv')
iris_df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [2]:
iris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
Id               150 non-null int64
SepalLengthCm    150 non-null float64
SepalWidthCm     150 non-null float64
PetalLengthCm    150 non-null float64
PetalWidthCm     150 non-null float64
Species          150 non-null object
dtypes: float64(4), int64(1), object(1)
memory usage: 7.1+ KB


### Splitting the data into feature set and target set

In [0]:
#X_iris = iris_df
#Y_iris = X_iris[['Species']]
#X_iris.drop(columns=['Species','Id'],inplace=True)

In [0]:
#X_iris.shape

In [5]:
iris_df = pd.get_dummies(iris_df,prefix=['Species'])
iris_df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0
3,4,4.6,3.1,1.5,0.2,1,0,0
4,5,5.0,3.6,1.4,0.2,1,0,0


In [6]:
iris_df.columns

Index([u'Id', u'SepalLengthCm', u'SepalWidthCm', u'PetalLengthCm',
       u'PetalWidthCm', u'Species_Iris-setosa', u'Species_Iris-versicolor',
       u'Species_Iris-virginica'],
      dtype='object')

In [0]:
y_iris = iris_df[['Species_Iris-setosa','Species_Iris-versicolor','Species_Iris-virginica']]
X_iris = iris_df.drop(['Id','Species_Iris-setosa','Species_Iris-versicolor','Species_Iris-virginica'],axis=1)

In [8]:
print(X_iris.shape)
print(y_iris.shape)

(150, 4)
(150, 3)


### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [9]:
'''from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Y_iris = pd.DataFrame(le.fit_transform(Y_iris))'''

'''Y_iris = pd.get_dummies(Y_iris)
Y_iris.head()'''

'Y_iris = pd.get_dummies(Y_iris)\nY_iris.head()'

In [0]:
#Y_iris.shape

### Divide the dataset into Training and test (70:30)

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_iris_train,X_iris_test,y_iris_train,y_iris_test = train_test_split(X_iris,y_iris,test_size=0.3)

In [0]:
#X_iris_train.shape

In [0]:
#X_iris_train.head()

In [0]:
#y_iris_test.head()

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [0]:
import tensorflow as tf1

tf1.reset_default_graph()

#Initialize Sequential model
model_iris = tf1.keras.models.Sequential()

#Reshape data from 2D to 1D -> 28x28 to 784
#model.add(tf.keras.layers.Reshape((784,),input_shape=(4,)))

#Normalize the data
model_iris.add(tf1.keras.layers.BatchNormalization(input_shape=(4,)))


#Add Dense Layer which provides 10 Outputs after applying softmax
model_iris.add(tf1.keras.layers.Dense(3))

#Comile the model
model_iris.compile(optimizer='sgd', loss='mse')



In [0]:
#Add 1st hidden layer
model_iris.add(tf1.keras.layers.Dense(10, activation='relu'))##use sigmoid instead of relu for this residency



In [0]:
#Add 2nd hidden layer
model_iris.add(tf1.keras.layers.Dense(8, activation='relu'))


In [0]:

#Add OUTPUT layer
model_iris.add(tf1.keras.layers.Dense(3, activation='softmax'))



In [0]:
#Create optimizer with non-default learning rate
sgd_optimizer = tf1.keras.optimizers.SGD(lr=0.03)

#Compile the model
model_iris.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [21]:
model_iris.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 4)                 16        
_________________________________________________________________
dense (Dense)                (None, 3)                 15        
_________________________________________________________________
dense_1 (Dense)              (None, 10)                40        
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 88        
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 27        
Total params: 186
Trainable params: 178
Non-trainable params: 8
_________________________________________________________________


### Fitting the model and predicting 

In [22]:
model_iris.fit(X_iris_train,y_iris_train,          
          validation_data=(X_iris_test,y_iris_test),
          epochs=30,batch_size=32)

Train on 105 samples, validate on 45 samples
Epoch 1/30
105/105 [==============================] - 0s 3ms/step - loss: 1.5597 - acc: 0.2476 - val_loss: 2.2841 - val_acc: 0.2889
Epoch 2/30
105/105 [==============================] - 0s 116us/step - loss: 1.2076 - acc: 0.2476 - val_loss: 2.0222 - val_acc: 0.2889
Epoch 3/30
105/105 [==============================] - 0s 140us/step - loss: 1.0407 - acc: 0.2667 - val_loss: 1.8625 - val_acc: 0.2889
Epoch 4/30
105/105 [==============================] - 0s 116us/step - loss: 0.9557 - acc: 0.5048 - val_loss: 1.7767 - val_acc: 0.2889
Epoch 5/30
105/105 [==============================] - 0s 129us/step - loss: 0.9108 - acc: 0.5429 - val_loss: 1.7089 - val_acc: 0.2889
Epoch 6/30
105/105 [==============================] - 0s 116us/step - loss: 0.8694 - acc: 0.6476 - val_loss: 1.6538 - val_acc: 0.4667
Epoch 7/30
105/105 [==============================] - 0s 129us/step - loss: 0.8294 - acc: 0.6667 - val_loss: 1.5753 - val_acc: 0.4222
Epoch 8/30
105/105 

### Report Accuracy of the predicted values

In [23]:
scores = model_iris.evaluate(X_iris_test,y_iris_test)
scores

45/45 [==============================] - 0s 165us/step


[0.4378734416431851, 0.7111111177338494]